In [1]:
!pip install pyagrum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 22.7 MB/s eta 0:00:00


In [2]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

In [3]:
#Creamos la red bn con el nombre 'Rendimiento academico'
bn=gum.BayesNet("Rendimiento academico")

#Creamos los nodos como variables
ns,er,pec,br,ae=[bn.add(name,2) for name in("Nivell de son","Estudi regular","participacio en classe","Bon rendiment","Activitats extraescolars")]

#Creamos los arcos
for arc in[(er,br),(ae,br),(pec,br),(ns,er),(er,pec)]:
  bn.addArc(arc[0],arc[1])


#Rellenamos las tablas de probabilidad condicional (CPT) para poder hacer la inferencia
bn.cpt(ns).fillWith([0.2,0.8])
bn.cpt(ae).fillWith([0.7,0.3])

bn.cpt(er)[0]=[0.6,0.4]
bn.cpt(er)[1]=[0.3,0.7]

bn.cpt(pec)[0]=[0.8,0.2]
bn.cpt(pec)[1]=[0.3,0.7]

bn.cpt(br)[0,0,0] = [0.9, 0.1]
bn.cpt(br)[0,0,1] = [0.5, 0.5]
bn.cpt(br)[0,1,0] = [0.7, 0.3]
bn.cpt(br)[0,1,1] = [0.3, 0.7]
bn.cpt(br)[1,0,0] = [0.6, 0.4]
bn.cpt(br)[1,0,1] = [0.2, 0.8]
bn.cpt(br)[1,1,0] = [0.4, 0.6]
bn.cpt(br)[1,1,1] = [0.1, 0.9]


#Número de dígitos en las tablas de probabilidad
gum.config['notebook','potential_visible_digits'] = 2

#Mostramos la red y las CPT
gnb.sideBySide(bn.cpt(ns),bn.cpt(er),captions=['P(Nivell de son)','P(Estudi regular|Nivell son)']) # 1a fila
gnb.sideBySide(bn.cpt(ae),gnb.getBN(bn,size="5!"),bn.cpt(pec),captions=['P(Activitats extraescolars)',"",'P(Participacio en classe|Estudi regular)']) # 2a fila
gnb.sideBySide(bn.cpt(br),captions=['P(Bon rendiment|Estudi regular,Activitats extraescolars,Participacio classe)']) # 3a fila



Quina és la probabilitat que un estudiant tingui un bon rendiment acadèmic si sabem que participa en activitats extraescolars i té un bon nivell de son?

In [4]:
ie=gum.LazyPropagation(bn)
gum.config['notebook','potential_visible_digits']=4
ie.setEvidence({ae:1,ns:1})
ie.makeInference()
#Probabilidades a posterior de saber que ae=1 y ns=1
posterior=ie.posterior(br)
posterior

(pyAgrum.Potential<double>@0x56c347746af0) 
  Bon rendiment    |
0        |1        |
---------|---------|
 0.3040  | 0.6960  |

Si un estudiant no té un bon nivell de son, quina és la probabilitat que tingui un bon rendiment acadèmic, sabent que estudia de manera regular?

In [5]:
ie=gum.LazyPropagation(bn)
gum.config['notebook','potential_visible_digits']=4
ie.setEvidence({er:1,ns:0})
ie.makeInference()
#Probabilidades a posterior de saber que er=1 y ns=0
posterior=ie.posterior(br)
posterior

(pyAgrum.Potential<double>@0x56c346b45680) 
  Bon rendiment    |
0        |1        |
---------|---------|
 0.2510  | 0.7490  |

Quina és la probabilitat que un estudiant tingui un nivell de son adequat si sabem que té un bon rendiment acadèmic?

In [6]:
ie=gum.LazyPropagation(bn)
gum.config['notebook','potential_visible_digits']=4
ie.setEvidence({er:1})
ie.makeInference()
#Probabilidades a posterior de saber ns=1
posterior=ie.posterior(ns)
posterior

(pyAgrum.Potential<double>@0x56c346fc0fa0) 
  Nivell de son    |
0        |1        |
---------|---------|
 0.1250  | 0.8750  |

Si un estudiant participa en activitats extraescolars, quina és la probabilitat que estudiï de manera regular?

In [7]:
ie=gum.LazyPropagation(bn)
gum.config['notebook','potential_visible_digits']=4
ie.setEvidence({ae:1})
ie.makeInference()
#Probabilidades a posterior de saber que ae=1
posterior=ie.posterior(er)
posterior

(pyAgrum.Potential<double>@0x56c3477cd7d0) 
  Estudi regular   |
0        |1        |
---------|---------|
 0.3600  | 0.6400  |

Quina és la probabilitat que un estudiant tingui un bon rendiment acadèmic si sabem que no participa en classe i no té un bon nivell de son?

In [8]:
ie=gum.LazyPropagation(bn)
gum.config['notebook','potential_visible_digits']=4
ie.setEvidence({pec:0,ns:0})
ie.makeInference()
#Probabilidades a posterior de saber que pec=0 y ns=0
posterior=ie.posterior(br)
posterior

(pyAgrum.Potential<double>@0x56c34797ac80) 
  Bon rendiment    |
0        |1        |
---------|---------|
 0.7600  | 0.2400  |